In [24]:
! lamin load scprint

^C
Traceback (most recent call last):
  File "/home/ml4ig1/miniconda3/envs/scprint/bin/lamin", line 5, in <module>
    from lamin_cli.__main__ import main
  File "/home/ml4ig1/miniconda3/envs/scprint/lib/python3.10/site-packages/lamin_cli/__main__.py", line 11, in <module>
    import rich_click as click
  File "/home/ml4ig1/miniconda3/envs/scprint/lib/python3.10/site-packages/rich_click/__init__.py", line 73, in <module>
    from . import rich_click as rich_click
  File "/home/ml4ig1/miniconda3/envs/scprint/lib/python3.10/site-packages/rich_click/rich_click.py", line 6, in <module>
    import rich.columns
  File "/home/ml4ig1/miniconda3/envs/scprint/lib/python3.10/site-packages/rich/columns.py", line 7, in <module>
    from .console import Console, ConsoleOptions, RenderableType, RenderResult
  File "/home/ml4ig1/miniconda3/envs/scprint/lib/python3.10/site-packages/rich/console.py", line 57, in <module>
    from .markup import render as render_markup
  File "<frozen importlib._bootstra

In [1]:
from lightning.pytorch import Trainer, seed_everything
from lightning.pytorch.callbacks import ModelCheckpoint, StochasticWeightAveraging, EarlyStopping, LearningRateMonitor, LearningRateFinder

seed_everything(42, workers=True)

from scprint import scPrint
from scprint.trainer import TrainingMode
from scdataloader import DataModule 
import pandas as pd
from scdataloader.utils import load_genes

import torch
torch.set_float32_matmul_precision('medium')

%load_ext autoreload
%autoreload 2

Global seed set to 42


💡 connected lamindb: jkobject/scprint


/home/ml4ig1/miniconda3/envs/scprint/lib/python3.10/site-packages/umap/__init__.py:9: ImportWarning: Tensorflow not installed; ParametricUMAP will be unavailable
  warn(
2024-05-14 10:34:41,093:INFO - Downloading data from `https://omnipathdb.org/queries/enzsub?format=json`
2024-05-14 10:34:41,233:INFO - Downloading data from `https://omnipathdb.org/queries/interactions?format=json`
2024-05-14 10:34:41,327:INFO - Downloading data from `https://omnipathdb.org/queries/complexes?format=json`
2024-05-14 10:34:41,420:INFO - Downloading data from `https://omnipathdb.org/queries/annotations?format=json`
2024-05-14 10:34:41,506:INFO - Downloading data from `https://omnipathdb.org/queries/intercell?format=json`
2024-05-14 10:34:41,716:INFO - Downloading data from `https://omnipathdb.org/about?format=text`
/home/ml4ig1/miniconda3/envs/scprint/lib/python3.10/site-packages/arboreto/core.py:12: DeprecationWarning: The current Dask DataFrame implementation is deprecated. 
In a future release, Dask D

In [2]:
# TODO: drop tissue & dev stage until part or is taken in account

hierarchical_clss = [
    "cell_type_ontology_term_id", #1
    # "tissue_ontology_term_id",
    "disease_ontology_term_id", # 2
#    "development_stage_ontology_term_id",
    "assay_ontology_term_id", #3
    'self_reported_ethnicity_ontology_term_id', #4
]
clss_to_pred = hierarchical_clss+[
    'sex_ontology_term_id', #5
    "organism_ontology_term_id", #6
]
all_clss = clss_to_pred+[
    #'dataset_id',
    #'cell_culture',
  #  "heat_diff",
  #  "total_counts",
  # "nnz",
  #  "dpt_group",
]

gene_emb = '../data/main/gene_embeddings.parquet'
d_model=128

In [3]:
datamodule = DataModule(
    collection_name="some",
    gene_embeddings=gene_emb,
    all_clss=all_clss,
    hierarchical_clss=hierarchical_clss,
    organisms=["NCBITaxon:9606"],#, "NCBITaxon:10090"],
    how="most expr",
    max_len=1200,
    add_zero_genes=0,
    # how much more you will see the most present vs less present category 
    weight_scaler=10,
    clss_to_weight=clss_to_pred,
    clss_to_pred=clss_to_pred,
    batch_size=1,
    num_workers=1,
    #train_oversampling=2,
    validation_split=0.05,
    do_gene_pos='../data/main/biomart_pos.parquet',
    test_split=0.05)
testfiles = datamodule.setup()

won't do any check but we recommend to have your dataset coming from local storage

80.0% are aligned
seeing a string: loading gene positions as biomart parquet file


In [4]:
# create a function to transform an scGPT checkpoint to an scPrint's
# ckpt = torch.load("../../scGPT/save/model_e6.pt")
# scPrint.load_from_checkpoint("../../scGPT/save/model_e6.pt")

In [5]:
#from lightning.pytorch.profilers import PyTorchProfiler
#pytorch_prof = PyTorchProfiler("../data/tensorboard", emit_nvtx=False, group_by_input_shape=True, record_shapes=True, profile_memory=True, with_stack=True, on_trace_ready=torch.profiler.tensorboard_trace_handler("../data/tensorboard/"),)

In [6]:
d_model

128

In [4]:
model = scPrint(
    genes = datamodule.genes,
    d_model = d_model*4,
    nhead = 4*2,
    nlayers = 1,
    #layers_cls = [d_model],
    #labels = datamodule.labels,
    #cls_hierarchy = datamodule.cls_hierarchy,
    dropout= 0,
    transformer = "flash",
    precpt_gene_emb = gene_emb,
    gene_pos_enc = datamodule.gene_pos,
    mvc_decoder = "inner product",
    #label_decoders = datamodule.decoders,
    fused_dropout_add_ln = False,
    #num_batch_labels = datamodule.num_datasets,
    checkpointing=False,
    prenorm=True,
    num_heads_kv=None,
    weight_decay=0,
)

In [5]:
#from lightning.pytorch.loggers import TensorBoardLogger
from lightning.pytorch.loggers import WandbLogger

wandb_logger = WandbLogger(project="scprint_test", save_dir="../data/tensorboard")
wandb_logger.watch(model, log='all', log_freq=50, log_graph=True)

#tlogger = TensorBoardLogger(save_dir="../data/tensorboard")
#tlogger.log_graph(model)


2024-05-14 10:35:18,565:ERROR - Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: jkobject (ml4ig). Use `wandb login --relogin` to force relogin
/home/ml4ig1/miniconda3/envs/scprint/lib/python3.10/site-packages/wandb/sdk/lib/ipython.py:77: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import HTML, display  # type: ignore


wandb: logging graph, to disable use `wandb.watch(log_graph=False)`


In [6]:
chckp = ModelCheckpoint(monitor="val_loss", save_top_k=-1)
trainingmode = TrainingMode(
    do_denoise=True, 
    noise=[0.01],
    do_cce=False, 
    cce_sim=0.6, 
    do_ecs=False, 
    ecs_threshold = 0.4, 
    ecs_scale = 0.05,
    class_scale = 0.08,
    do_cls=False,
    do_mvc=False, 
    do_adv_cls=False,
    do_next_tp=False, 
    mask_ratio=[], 
    warmup_duration= 100, 
    fused_adam= True,
    lr_reduce_patience=200,
    )
#es = EarlyStopping(patience=2, monitor='val_loss')
#swa = StochasticWeightAveraging(swa_lrs= 0.01)
#lrm = LearningRateMonitor(logging_interval="step")
#lrf = LearningRateFinder(mode="exponential",)
# TODO: to check that the class hierarchy are really ordered from 1-2-3-4... as well (oredered dict)
trainer = Trainer(precision="16-mixed", gradient_clip_val=500, max_time={"hours": 2}, limit_val_batches=1, callbacks=[trainingmode], accumulate_grad_batches=1, check_val_every_n_epoch=10000000,reload_dataloaders_every_n_epochs=1000000, logger=wandb_logger)#, logger=tlogger) #detect_anomaly=True, fast_dev_run=20, overfit_batches=10, limit_train_batches=1, limit_val_batches=0
#logger=wandb_logger,

Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
`Trainer(limit_val_batches=1)` was configured so 1 batch will be used.


In [7]:
# sanity. should be overfiting.
trainer = Trainer(precision="16-mixed", gradient_clip_val=500, max_time={"hours": 2}, limit_val_batches=1, callbacks=[trainingmode], accumulate_grad_batches=1, check_val_every_n_epoch=1000_000, overfit_batches=1,reload_dataloaders_every_n_epochs=1000_000, logger=wandb_logger)

Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
`Trainer(overfit_batches=1)` was configured so 1 batch will be used.


In [8]:
model.lr_reduce_patience = 50
model.lr_reduce_ratio = 0.2

In [9]:
trainer.fit(model, datamodule=datamodule)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type                    | Params
----------------------------------------------------------
0 | gene_encoder  | GeneEncoder             | 11.9 M
1 | expr_encoder  | ContinuousValueEncoder  | 264 K 
2 | pos_encoder   | PositionalEncoding      | 0     
3 | class_encoder | CategoryValueEncoder    | 512   
4 | depth_encoder | ContinuousValueEncoder  | 264 K 
5 | transformer   | FlashTransformerEncoder | 3.2 M 
6 | expr_decoder  | ExprDecoder             | 528 K 
7 | cls_decoders  | ModuleDict              | 0     
8 | mvc_decoder   | MVCDecoder              | 1.1 M 
----------------------------------------------------------
5.3 M     Trainable params
11.9 M    Non-trainable params
17.1 M    Total params
68.459    Total estimated model params size (MB)
wandb: WARNING Serializing object of type list that is 194680 bytes
wandb: WARNING Serializing object of type list that is 194680 bytes
wandb: WARNING Serializing object of type l

Sanity Checking: 0it [00:00, ?it/s]

> /home/ml4ig1/Documents code/scPRINT/scprint/model/model.py(852)_compute_loss()
    850         import/home/ml4ig1/Documents code/scPRINT/scprint/model/model.py(852)_compute_loss()
    850         import pdb
    851         pdb.set_trace()
--> 852         loss_expr = loss.zinb(
    853             theta=output["disp"],
    854             pi=output["zero_logits"],

*** NameError: name 'F' is not defined
*** NameError: name 'F' is not defined
tensor([[238., 141., 138.,  ...,   1.,   1.,   1.]], device='cuda:0')
tensor([[238., 141., 138.,  ...,   0.,   0.,   0.]], device='cuda:0')
tensor(2.8642, device='cuda:0')
tensor([[-10., -10., -10.,  ..., -10., -10., -10.]], device='cuda:0')
*** NameError: name 'disp' is not defined
tensor([[100., 100., 100.,  ..., 100., 100., 100.]], device='cuda:0')
tensor(1.1938, device='cuda:0')
tensor(1.1816, device='cuda:0')
tensor(nan, device='cuda:0')
tensor(3.0091, device='cuda:0')
tensor(7.2273, device='cuda:0')
tensor(1.1851, device='cuda:0')
tensor(1.2

In [ ]:
expression[:100]

In [ ]:
weirdly, when input expression is set to 0 and pi is set to 1000. I get a loss of 0
when input expression is set to some set of values and pi=-10, disp=max, mean=input, I get 0.479, decreasing pi doesn't help, making it higher, makes it worse, max of disp is ~3M afterward it increases back (likely due to some overflow??). I am guessing that it would need to be set to infinity for it to work. might be better to use dispersion instead of inverse dispersion?

all things equal, if disp is set to 2_700_000, I get loss tensor(3.8694, device='cuda:0')
if disp is 2_700_000+1 -> tensor(0.1994, device='cuda:0')

basically it seems that at too high values or at too low values we get strong instabilities that are exacerbated by the fact that we are in float16.
could it also be because of the espilon? nope

-> why doesn't my model go to this low loss value??



In [ ]:
import torch.nn.functional as F

In [ ]:
F.mse_loss(expression, expression, reduction="mean")

In [ ]:
loss.nb(
    theta=disp,
    mu=expression,
    target=expression,
)

In [ ]:
loss.nb(
    theta=output["disp"],
    mu=output["mean"],
    target=expression,
).mean()

In [ ]:
loss.zinb(
            theta=output["disp"],
            pi=output["zero_logits"],
            mu=output["mean"],
            target=expression,
        )

In [ ]:
exp(15)

In [ ]:
disp = torch.zeros_like(output["disp"])
pi = torch.zeros_like(output["disp"]) - 10

In [ ]:
# why if it is the inverse dispersion? does setting it to 3_000_000 works but not 10_000_000?
# maybe some overflow??

In [ ]:
loss.zinb(
    theta=disp,
    pi=pi,
    mu=output["mean"],
    target=expression,
)

In [ ]:
loss.zinb(
    theta=disp,
    pi=pi,
    mu=output["mean"],
    target=expression,
)

In [ ]:
loss.zinb(
    theta=disp,
    pi=pi,
    mu=expression,
    target=expression,
)

In [ ]:
loss.zinb(
    theta=output["disp"],
    pi=output["zero_logits"],
    mu=expression,
    target=expression,
)

In [ ]:
loss.zinb(
    theta=disp,
    pi=pi,
    mu=expression,
    target=expression,
)

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir="../data/tensorboard"

In [ ]:
#wandb_logger.finalize(status="aborted")
torch.cuda.empty_cache()